In [1]:
try:
    import google.colab  # noqa: F401

    # specify the version of DataEval (==X.XX.X) for versions other than the latest
    %pip install -q dataeval
except Exception:
    pass

In [2]:
import numpy as np

from dataeval.metrics.estimators import ber
from dataeval.utils.data.datasets import MNIST

In [3]:
# Load in both the training and testing mnist dataset
train_ds = MNIST(root="./data/", train=True, download=True, flatten=True)
test_ds = MNIST(root="./data/", train=False, download=True, channels="channels_last")

# Split out the images and labels for each set
images, labels = train_ds.data, train_ds.targets
test_images, test_labels = test_ds.data, test_ds.targets

Files already downloaded and verified


Files already downloaded and verified


In [4]:
print("Number of training samples: ", len(images))
print("Image shape:", images[0].shape)
print("Label counts: ", np.unique(labels, return_counts=True))

Number of training samples:  54210
Image shape: (784,)
Label counts:  (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([5421, 5421, 5421, 5421, 5421, 5421, 5421, 5421, 5421, 5421]))


In [5]:
images_split = {}
labels_split = {}

# Keep only 1, 4, and 9
for label in (1, 4, 9):
    subset_indices = np.where(labels == label)
    images_split[label] = images[subset_indices][:2000]
    labels_split[label] = labels[subset_indices][:2000]

images_subset = np.concatenate(list(images_split.values()))
labels_subset = np.concatenate(list(labels_split.values()))
print(images_subset.shape)
print(np.unique(labels_subset, return_counts=True))

(6000, 784)
(array([1, 4, 9], dtype=uint8), array([2000, 2000, 2000]))


In [6]:
# Evaluate the BER metric for the MNIST data with digits 1, 4, 9.
# One minus the value of this metric gives our estimate of the upper bound on accuracy.
base_result = ber(images_subset, labels_subset, method="MST")

In [7]:
print("The bayes error rate estimation:", base_result.ber)

The bayes error rate estimation: 0.022833333333333334


In [8]:
### TEST ASSERTION CELL ###
assert 0.976 < 1 - base_result.ber < 0.978

In [9]:
print("The maximum achievable accuracy:", 1 - base_result.ber)

The maximum achievable accuracy: 0.9771666666666666


In [10]:
# Creates a binary mask where current label == 1 that can be used as the new labels
labels_merged = labels_subset == 1
print("New label counts:", np.unique(labels_merged, return_counts=True))

New label counts: (array([False,  True]), array([4000, 2000]))


In [11]:
# Evaluate the BER metric for the MNIST data with updated labels
new_result = ber(images_subset, labels_merged, method="MST")

In [12]:
print("The bayes error rate estimation:", new_result)

The bayes error rate estimation: BEROutput: {'ber': np.float64(0.005333333333333333), 'ber_lower': np.float64(0.002673815958446346)}


In [13]:
### TEST ASSERTION CELL ###
assert 0.994 < 1 - new_result.ber < 0.996

In [14]:
print("The maximum achievable accuracy:", 1 - new_result.ber)

The maximum achievable accuracy: 0.9946666666666667
